In [5]:
import csv
import os
import pandas as pd

# Fixing Overtime Game Columns

### Loop Code

In [ ]:
### Functions to loop through them

# Function to add 'OT' and 'OT_y_n' columns
def add_ot_columns(input_path, output_path):
    with open(input_path, 'r') as infile, open(output_path, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        header = next(reader)
        corrected_header = header[:5] + ['OT'] + header[5:9] + ['OT_y_n'] + header[9:]
        writer.writerow(corrected_header)

        for row in reader:
            corrected_row = row[:5] + [''] + row[5:9] + [''] + row[9:]
            writer.writerow(corrected_row)

# Function to shift values in overtime games
def shift_values(input_path, output_path):
    with open(input_path, 'r') as infile, open(output_path, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        header = next(reader)
        writer.writerow(header)

        column_mapping = {
            'OT_y_n': 'PFR_ID',
            'PFR_ID': 'Game_ID',
            'Game_ID': 'Total',
            'Total': 'OT'
        }

        for row in reader:
            remapped_row = row.copy()

            if len(row) >= 10 and row[9]:
                for old_col, new_col in column_mapping.items():
                    old_index = header.index(old_col)
                    new_index = header.index(new_col)
                    remapped_row[new_index] = row[old_index]

                remapped_row[header.index('OT_y_n')] = 'y'
            else:
                remapped_row[header.index('OT_y_n')] = 'n'

            writer.writerow(remapped_row)


In [ ]:
### Loop through them all

# Iterate through each file in the input directory
for filename in os.listdir('./data-raw-sr/box-scores/'):
    if filename.endswith('.csv'):
        # Step 1: Add 'OT' and 'OT_y_n' columns
        add_ot_columns(f'./data-raw-sr/box-scores/{filename}', f'./data-raw-sr/box-scores/intermediate_{filename}')
        
        # Step 2: Shift values and mark 'OT_y_n' as 'y' or 'n'
        shift_values(f'./data-raw-sr/box-scores/intermediate_{filename}', f'./data-raw-sr/box-scores/cleaned_{filename}')

        # Optionally, remove the intermediate file
        os.remove(f'./data-raw-sr/box-scores/intermediate_{filename}')

print("Processed all CSV files. Final remapped files with 'OT_y_n' marked as 'y' or 'n' have been saved.")


In [ ]:
### Clean last column's commas

directory_path = './data-raw-sr/box-scores/'

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    
    # Check if the file is a CSV and starts with 'cleaned_'
    if filename.endswith('.csv') and filename.startswith('cleaned_'):
        
        input_file_path = os.path.join(directory_path, filename)
        output_file_path = os.path.join(directory_path, f"v2_{filename}")
        
        # Open the input file for reading and the output file for writing
        with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
            
            # Loop through each line in the input file
            for line in infile:
                
                # Remove trailing commas and write to the output file
                outfile.write(line.rstrip(",\n") + "\n")


In [ ]:
### Delete old versions



# Combining Games to Single Rows

In [ ]:
# ### Single year

# # Initialize an empty DataFrame to store the combined rows
# combined_df = pd.DataFrame()

# # Group the DataFrame by 'Game_ID' and aggregate the rows into lists
# nfl_df = pd.read_csv('./data-raw-sr/box-scores/v2_cleaned_all_nfl_box_scores_2022.csv')
# grouped_df = nfl_df.groupby('Game_ID').agg(list).reset_index()

# # Loop to append data to the DataFrame
# for idx, row in grouped_df.iterrows():
#     combined_data = {'Game_ID': row['Game_ID']}
#     for col in nfl_df.columns:
#         if col != 'Game_ID':
#             combined_data[f'Team1_{col}'] = row[col][0]
#             combined_data[f'Team2_{col}'] = row[col][1]
    
#     # Create a DataFrame for the new row and concatenate it with the existing DataFrame
#     new_row_df = pd.DataFrame([combined_data])
#     combined_df = pd.concat([combined_df, new_row_df], ignore_index=True)

# # ... rest of your code, like abbreviation extraction and saving to CSV
# # Extract team abbreviations from 'Game_ID'
# combined_df['Team1_ABBR'] = combined_df['Game_ID'].str.extract(r'\d+_\d+_(\w+)_\w+$')[0]
# combined_df['Team2_ABBR'] = combined_df['Game_ID'].str.extract(r'\d+_\d+_\w+_(\w+)$')[0]

# # Drop duplicate columns related to 'PFR_ID' and 'OT_y_n' for Team2
# combined_df.drop(columns=['Team2_PFR_ID', 'Team2_OT_y_n'], inplace=True)

# # Rename the remaining 'PFR_ID' and 'OT_y_n' columns
# combined_df.rename(columns={'Team1_PFR_ID': 'PFR_ID', 'Team1_OT_y_n': 'OT_y_n'}, inplace=True)

# # Save the combined DataFrame to a new CSV file
# combined_df.to_csv('./data-raw-sr/box-scores/v3_cleaned_all_nfl_box_scores_2022.csv', index=False)


In [ ]:
### Loop it

# Directory containing your data files
data_dir = './data-raw-sr/box-scores/'

# Loop through each CSV file in the directory
for filename in os.listdir(data_dir):
    if filename.startswith('v2_') and filename.endswith('.csv'):
        
        # Construct the full path of the file
        file_path = os.path.join(data_dir, filename)
        
        # Initialize an empty DataFrame to store the combined rows
        combined_df = pd.DataFrame()

        # Read and group the DataFrame by 'Game_ID'
        nfl_df = pd.read_csv(file_path)
        grouped_df = nfl_df.groupby('Game_ID').agg(list).reset_index()

        # Loop to append data to the DataFrame
        for idx, row in grouped_df.iterrows():
            combined_data = {'Game_ID': row['Game_ID']}
            for col in nfl_df.columns:
                if col != 'Game_ID':
                    combined_data[f'Team1_{col}'] = row[col][0]
                    combined_data[f'Team2_{col}'] = row[col][1]

            new_row_df = pd.DataFrame([combined_data])
            combined_df = pd.concat([combined_df, new_row_df], ignore_index=True)
        
        # Extract team abbreviations from 'Game_ID'
        combined_df['Team1_ABBR'] = combined_df['Game_ID'].str.extract(r'\d+_\d+_(\w+)_\w+$')[0]
        combined_df['Team2_ABBR'] = combined_df['Game_ID'].str.extract(r'\d+_\d+_\w+_(\w+)$')[0]

        # Drop duplicate columns related to 'PFR_ID' and 'OT_y_n' for Team2
        combined_df.drop(columns=['Team2_PFR_ID', 'Team2_OT_y_n'], inplace=True)

        # Rename the remaining 'PFR_ID' and 'OT_y_n' columns
        combined_df.rename(columns={'Team1_PFR_ID': 'PFR_ID', 'Team1_OT_y_n': 'OT_y_n'}, inplace=True)

        # Save the combined DataFrame to a new CSV file
        # Construct a new filename to save the processed data
        new_filename = f'v3_cleaned_{filename}'
        new_file_path = os.path.join(data_dir, new_filename)
        
        combined_df.to_csv(new_file_path, index=False)

print("All files processed.")


# Giving Standard Abbreviations and Game_ID's

In [3]:
# ### Adding Team1_ABBR_Standard and Team2_ABBR_Standard columns for single year
# ### Need to switch from final directory

# # Load the CSV file into a DataFrame
# #file_path = './data-raw-sr/box-scores/box_scores_with_scorers_2021.csv'
# file_path = './data-final/box_scores_with_scorers_2021.csv'
# df = pd.read_csv(file_path)

# # Create a mapping dictionary for the teams that got renamed or relocated
# team_name_mapping = {
#     'STL': 'LAR',  # St. Louis Rams to Los Angeles Rams
#     'OAK': 'LVR',  # Oakland Raiders to Las Vegas Raiders
#     'LV': 'LVR',   # LV to LVR just cause
#     'SD': 'LAC',   # San Diego Chargers to Los Angeles Chargers
#     'LA': 'LAR'    # LA could refer to Los Angeles Rams
# }

# # Update the team abbreviations and create new columns for them
# df['Team1_ABBR_Standard'] = df['Team1_ABBR'].map(lambda x: team_name_mapping.get(x, x))
# df['Team2_ABBR_Standard'] = df['Team2_ABBR'].map(lambda x: team_name_mapping.get(x, x))

# # Save the updated DataFrame back to a CSV file
# #output_file_path = './data-raw-sr/box-scores/box_scores_with_scorers_2021_updated.csv'
# output_file_path = './data-final/box_scores_with_scorers_2021_updated.csv'
# df.to_csv(output_file_path, index=False)


In [4]:
# ### Adding Game_ID_Standard column for single year
# ### Need to switch from final directory

# # Load the existing CSV file into a DataFrame
# #file_path = './data-raw-sr/box-scores/box_scores_with_scorers_2021.csv'
# file_path = './data-final/box_scores_with_scorers_2021_updated.csv'
# df = pd.read_csv(file_path)

# # Create the 'Game_ID_Standard' column
# df['Game_ID_Standard'] = df.apply(
#     lambda row: f"{row['Game_ID'].split('_')[0]}_{row['Game_ID'].split('_')[1]}_{row['Team1_ABBR_Standard']}_{row['Team2_ABBR_Standard']}",
#     axis=1
# )

# # Save the updated DataFrame back to a new CSV file
# #output_file_path = './data-raw-sr/box-scores/box_scores_with_scorers_2021_updated.csv'
# output_file_path = './data-final/box_scores_with_scorers_2021_updated.csv'
# df.to_csv(output_file_path, index=False)


In [6]:
### Loop it all
### Need to switch from final directory

# Define the directory containing the CSV files
directory_path = './data-final/'

# Create a mapping dictionary for the teams that got renamed or relocated
team_name_mapping = {
    'STL': 'LAR',  # St. Louis Rams to Los Angeles Rams
    'OAK': 'LVR',  # Oakland Raiders to Las Vegas Raiders
    'LV': 'LVR',   # LV to LVR just cause
    'SD': 'LAC',   # San Diego Chargers to Los Angeles Chargers
    'LA': 'LAR'    # LA could refer to Los Angeles Rams
}

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    # Check if the file is a CSV and starts with 'box_scores_with_scorers'
    if filename.endswith('.csv') and filename.startswith('box_scores_with_scorers'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the existing CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Update the team abbreviations and create new columns for them
        df['Team1_ABBR_Standard'] = df['Team1_ABBR'].map(lambda x: team_name_mapping.get(x, x))
        df['Team2_ABBR_Standard'] = df['Team2_ABBR'].map(lambda x: team_name_mapping.get(x, x))
        
        # Create the 'Game_ID_Standard' column
        df['Game_ID_Standard'] = df.apply(
            lambda row: f"{row['Game_ID'].split('_')[0]}_{row['Game_ID'].split('_')[1]}_{row['Team1_ABBR_Standard']}_{row['Team2_ABBR_Standard']}",
            axis=1
        )
        
        # Save the updated DataFrame back to the same CSV file
        output_file_path = os.path.join(directory_path, filename)
        df.to_csv(output_file_path, index=False)
